# [Key-value memories](https://arxiv.org/pdf/1606.03126.pdf)
## Overview
New thing: using keys to address memories instead of the values.
Summary: for a given question, we preselect a subset of the words via hashing. Then, we repeat the following step several times "we calculate the similarity between the keys and the previous query, and use that to get a new query." (Note that in the first iteration, previous query == question.) Finally, we use the last memory to select the word that is most similar to it, and that's our answer.

## First step
Key hashing: we preselect all the memories whose keys share at least one word with the question. There's a caveat: we don't take into account common words like the, a, etc. So in the next equations, we are only dealing with a subset of the words given by the hash.

### Key addressing
We first transform the keys and the question to a better suited space. Then, we calculate the similarity between them. Finally, we take the softargmax of the distribution.

$$
p = softargmax(similarity(transform(keys), transform(question))) \\
similarity(x, y) = x \odot y \\
transform(k) = affine(bag\_of\_words_K(k)) = A\phi_K(k) \\
transform(q) = affine(bag\_of\_words_X(q)) = A\phi_Q(q)
$$

{why do they use the same transformation A for both k and q?}

### Reading
Let's call V to the list of memory vectors. We read each value vector by how similar are the values's key and the question.

$$o = attention(transform(v), weights=p) = \sum_i A\phi_V(v_i)p_i$$

## Memory steps
In the first step, we compute q_1 by mixing o and q_0 (the question.) 

In the second step, we compute o in terms of q_1. And q_2 is a mix of the new o and q_1.

We repeat the above process several times to deal with compositon.

$$q_{i+1} = R_{i+1} (o + q_i)$$

## Last step
We want the word that is most similar to the last memory.

$$
argmax_i(similarity(q_H, transform(y_i))) \\
similarity(x, y) = x^Ty \\
transform(y) = affine(bag\_of\_words_Y(y)) = A\phi_Y(y)
$$

{this seems to be a good place for the kernel trick, for everything is a linear combination of the original memories}

## Results
KB > raw wikipedia > IE

It's encouraging that raw wikipedia > IE, but the difference between KB and Raw wikipedia is large (94% to 76%). But most of that loss (94% to 83%) is just due to representing the information in sentences instead of triples. (For instance, there could be a five-words sentence (Movie came out in year) instead of the triple (movie come_out_in year.) That's seems more difficult to deal with. For instance, how can we be sure that we are doing the right segmentation?)

## Other details
The values used for the features vectors in the paper are that $\phi_K$ is the first two values of a triple and $\phi_V$ is the last value of the triple. 

Data sparsity issues: in NLP, most of the sentences we see are new, and new words appear often. Also, there could be a lot of redundancy. This is (in part) why latent semantic analysis, principal component analysis, and word embeddings make sense.

coreference: Eg _Mary_ was going to the library because _she_ wanted to play (Mary and she are coreferring each other.)